# Full Demo of `CHAMPPY`

This notebooks demonstrates how to use the framework `CHAMPPY` (Charging and Mobility Profiles in Python) to create mobility and charging profiles from your mobility data. 

## 1. Import Required Libraries
Import all necessary libraries, including pandas and relevant modules from champpy.

In [ ]:
import pandas as pd
import os
import champpy

## 2. Load raw data and create raw mobility profiles
Load the example logbooks and vehicles from csv files and create raw mobility profiles from it using the `MobData` class. You can use your own data. Therefore you have to bring your data in the required format. Use the provided examples for logbooks and vehicles as a guide to get your data into the required format.

In [ ]:
# Load example raw logbook data from CSV
base_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
logbook_path = os.path.join(base_dir, "data", "example1", "t_logbook.csv")
print(logbook_path)
raw_logbook_df = pd.read_csv(logbook_path, parse_dates=["dep_dt", "arr_dt"])

# Load example raw vehicle data from CSV
vehicle_path = os.path.join(base_dir, "data", "example1", "t_vehicle.csv")
raw_vehicle_df = pd.read_csv(vehicle_path, parse_dates=["first_day", "last_day"],date_format="%d-%b-%Y")

# Create raw mobility profiles
raw_profiles = champpy.MobData(input_logbooks_df=raw_logbook_df, input_vehicles_df=raw_vehicle_df)

## 3. Clean raw mobility profiles
To prepare your data for the next steps, you must clean the data. The `MobDataCleaner` class is provided for this purpose. It can be initialized by different user parameters using the `UserParamsCleaning` data class. After the cleaning, a summary is provided showing which data has been changed.

In [ ]:
# Initialize the data cleaner with user parameters
user_params_cleaning = champpy.UserParamsCleaning(
    speed = champpy.LimitConfig(min_value=0.01, min_method="delete", max_value=120.0, max_method="cap"),
	duration = champpy.LimitConfig(min_value=0.25, min_method="delete", max_value=8.0, max_method="cap"),
	distance = champpy.LimitConfig(min_value=0.5, min_method="delete", max_value=500.0, max_method="cap"),
	temp_res = 0.25,  # Temporal resolution in hours
	print_summary = True
)
data_cleaner = champpy.MobDataCleaner(user_params=user_params_cleaning)

# Apply data cleaner on your raw mobility profiles
mob_profile_cleaned = data_cleaner.clean(raw_profiles)

If you want to further validate the cleaning of the profiles, you can plot the mobility profiles before and after the cleaning using the `MobPlotter` class as shown below:

In [ ]:
# Create a copy of the cleaned mobility profiles and add the raw data for comparison using add_mob_data()
mob_data_merged = mob_profile_cleaned.copy()
mob_data_merged.add_mob_data(raw_profiles, old_cluster_label="Raw Data", new_cluster_label="Cleaned Data")

# Initialize the mobility plotter with user parameters for plotting
user_params_plot = champpy.UserParamsMobPlotter(
    filename="demo02_validation_data_cleaning.html",
    clustering =True,
    show=True,
    save_plot=True
)

# Create and instance of the mobility plotter and generate the plots for the merged mobility data
mobplot = champpy.MobPlotter(user_params_plot)
mobplot.plot_mob_data(mob_data_merged)

## 4. Parameterization
Parameterize the model based on your cleaned mobility profiles. The parameterization is performed using the `Paramterizer` class. It must be initialized by user parameters using the `UserParamsParameterizer` data class:

In [ ]:
# Define user params for the parameterization
user_params = champpy.UserParamsParameterizer(
    description="Example parameterization 1", # Define a description for the parameter set
    vehicle_type="Van", # Type of vehicle the parameters apply to (e.g., "Car", "Van", "Truck")
    temp_res = 0.25  # Temporal resolution in hours
)
# Create an instance of the Parameterizer
example_parameterizer = champpy.Parameterizer(user_params)

# calculate the model parameters based on the cleaned mobility profiles
model_params = example_parameterizer.calc_params(mob_profile_cleaned)

## 5. Generate Mobility Profiles
Using the `MobModel` and `UserParamsMobModel` to generate synthetic mobility profiles for a specified number of vehicles and date range. The model parameters generated in the previous step, serves as input to create the instance of `MobModel`.

In [ ]:
# Generate synthetic mobility profiles
mob_model = champpy.MobModel(model_params=model_params)
user_params_mob = champpy.UserParamsMobModel(
    number_vehicles=50,
    start_date=pd.Timestamp("2025-01-01-01:00:00"),
    end_date=pd.Timestamp("2025-12-31-23:00:00"),
)
mob_profiles = mob_model.generate_mob_profiles(user_params=user_params_mob)

## 6. Validate modeled mobility profiles
Merge the modeled mobility profiles with the cleaned reference profiles. Use the classes `UserParamsMobPlotter` and `MobPlotter` to create an html file that contains several validation plots.

In [ ]:
# Create a copy of the cleaned mobility profiles and add the modeld profiles for comparison using add_mob_data()
mob_data_merged = mob_profile_cleaned.copy()
mob_data_merged.add_mob_data(mob_profiles, old_cluster_label="Ref", new_cluster_label="Model")

# Initialize user parameters for plotting the mobiltiy profiles
user_params_plot = champpy.UserParamsMobPlotter(
    filename="demo02_mobility_profiles_validation_plot.html",
    clustering = True,
    show=True,
    save_plot=True
)

# Create instance of the mobility plotter
mobplot = champpy.MobPlotter(user_params_plot)

# Plot the mobility profiles for the merged data (ref + model)
mobplot.plot_mob_data(mob_data_merged)

## 7. Generate charging profiles
Use classes`ChargeModel` and `UserParamsChargeModel` to generate synthetic charging profiles.

In [ ]:
# Initilaize the charging model with the modeled mobility prfiles
charging_model = champpy.ChargingModel(mob_profiles)

# Define user parameters for the charging model
user_params_charging = champpy.UserParamsChargingModel(
    energy_consumption_kwh_per_km=[0.2],
    battery_capacity_kwh=[50.0],
    charging_power_max_kw=[11],
    efficiency_charging=[0.9],
    soc_min=[0.1],
    soc_min_dep=[0.8],
    soc_initial=1,
    distribute_energy_consumption=True,
    charging_locations=[1], 
    temp_res=0.25
)

# Generate charging profiles based on the mobility profiles and the user parameters for charging
charging_profiles = charging_model.generate_charging_profiles(user_params=user_params_charging)

## 8. Validate charging profiles

You can visulize the modeld charging profiles using the `ChargingPlotter` class. It must be initializes by different user parameters defined in the `UserParamsChargingPlotter` data class. 

In [ ]:
# Initialize user parameters for plotting the charging profiles
user_params_plot = champpy.UserParamsChargingPlotter(
    filename="demo02_charging_profiles_plot.html",
    show=True,
    save_plot=True,
    clustering=False
)

# Create an instance of the ChargingPlotter
chargeplot = champpy.ChargingPlotter(user_params_plot)

# Plot the charging profiles
chargeplot.plot_charging_profiles(charging_profiles)